In [1]:
import os
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import matplotlib.pyplot as plt
import json
import random
import image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable
import h5py
import numpy as np
import cv2
from utils import *
%matplotlib inline

/home/leeyh/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:127: RuntimeWarning: overflow encountered in add
  a = a + a
/home/leeyh/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:129: RuntimeWarning: invalid value encountered in subtract
  temp1 = temp - a
/home/leeyh/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:138: RuntimeWarning: invalid value encountered in subtract
  itemp = int_conv(temp-a)
/home/leeyh/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:162: RuntimeWarning: overflow encountered in add
  a = a + a
/home/leeyh/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:164: RuntimeWarning: invalid value encountered in subtract
  temp1 = temp - a
/home/leeyh/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:171: RuntimeWarning: invalid value encountered in subtract
  if any(temp-a != zero):


In [7]:
def gts_boxs(frame_size,gts):
    imgsize = (frame_size[1],frame_size[0])
    boxs = np.zeros((len(gts),4))
    for i in range(0,len(gts)):
        boxs[i] = np.array(region_to_bbox(gts[i]))
    return boxs
def init_video(root_dataset, dataset, video):
    video_folder = os.path.join(root_dataset, dataset, video)
    frame_name_list = [f for f in os.listdir(video_folder) if f.endswith(".jpg")]
    frame_name_list = [os.path.join(root_dataset, dataset, video, '') + s for s in frame_name_list]
    frame_name_list.sort()
    with Image.open(frame_name_list[0]) as img:
        frame_sz = np.asarray(img.size)
        frame_sz[1], frame_sz[0] = frame_sz[0], frame_sz[1]

    # read the initialization from ground truth
    gt_file = os.path.join(video_folder, 'groundtruth.txt')
    gt = np.genfromtxt(gt_file, delimiter=',')
    n_frames = len(frame_name_list)
    assert n_frames == len(gt), 'Number of frames and number of GT lines should be equal.'

    return gt, frame_name_list, frame_sz, n_frames
def region_to_bbox(region, center=True):
    n = len(region)
    assert n==4 or n==8, ('GT region format is invalid, should have 4 or 8 entries.')

    if n==4:
        return _rect(region, center)
    else:
        return _poly(region, center)
def _rect(region, center):
    
    if center:
        x = region[0]
        y = region[1]
        w = region[2]
        h = region[3]
        cx = x+w/2
        cy = y+h/2
        return cx, cy, w, h
    else:
        #region[0] -= 1
        #region[1] -= 1
        return region


def _poly(region, center):
    cx = np.mean(region[::2])
    cy = np.mean(region[1::2])
    x1 = np.min(region[::2])
    x2 = np.max(region[::2])
    y1 = np.min(region[1::2])
    y2 = np.max(region[1::2])
    A1 = np.linalg.norm(region[0:2] - region[2:4]) * np.linalg.norm(region[2:4] - region[4:6])
    A2 = (x2 - x1) * (y2 - y1)
    s = np.sqrt(A1/A2)
    w = s * (x2 - x1) + 1
    h = s * (y2 - y1) + 1

    if center:
        return cx, cy, w, h
    else:
        return cx-w/2, cy-h/2, w, h

# vot2017

In [19]:
root = '/home/leeyh/Downloads/vot2017/'

In [23]:
names = []
for name in glob.glob(os.path.join(root,'*')):
    names.append(name.replace(root,''))

In [24]:
names

['soldier',
 'bmx',
 'handball1',
 'zebrafish1',
 'birds1',
 'drone_across',
 'bolt2',
 'glove',
 'crabs1',
 'nature',
 'fernando',
 'soccer2',
 'drone1',
 'racing',
 'fish1',
 'road',
 'shaking',
 'fish2',
 'ball2',
 'gymnastics3',
 'dinosaur',
 'book',
 'hand',
 'bolt1',
 'iceskater2',
 'ants3',
 'tiger',
 'basketball',
 'pedestrian1',
 'matrix',
 'bag',
 'gymnastics2',
 'blanket',
 'leaves',
 'gymnastics1',
 'graduate',
 'godfather',
 'sheep',
 'iceskater1',
 'car1',
 'singer2',
 'helicopter',
 'traffic',
 'wiper',
 'frisbee',
 'motocross2',
 'ants1',
 'singer3',
 'crossing',
 'ball1',
 'rabbit',
 'conduction1',
 'drone_flip',
 'flamingo1',
 'motocross1',
 'girl',
 'soccer1',
 'handball2',
 'fish3',
 'butterfly']

In [25]:
root_dataset = '/home/leeyh/Downloads'
dataset = 'vot2017'
sequences_info = []
for name in names:
    gt, frame_name_list, frame_sz, n_frames = init_video(root_dataset, dataset, name)
    gts = gts_boxs(frame_sz,gt)
    track_ids = [[]]
    for i in xrange(len(gts)):
        box = gts[i]
        track_ids[0].append([frame_name_list[i],[float(box[0]-box[2]*0.5),float(box[0]+box[2]*0.5),float(box[1]-box[3]*0.5),float(box[1]+box[3]*0.5)]])
    sequences_info.append(track_ids)   

In [26]:
with open('vot2018.txt', 'w') as outfile:
    json.dump(sequences_info, outfile)

# otb100

In [9]:
root = '/home/leeyh/backup/data/'

In [11]:
names = []sequences_info.append(track_ids)
for name in glob.glob(os.path.join(root,'*')):
    names.append(name)

In [14]:
boxes = np.loadtxt(os.path.join(names[0],'groundtruth_rect.txt'),delimiter = ',')

In [51]:
sequences_info = []
for name in names:
    track_ids = [[]]
    try:
        boxes = np.loadtxt(os.path.join(name,'groundtruth_rect.txt'),delimiter = ',')
    except:
        boxes = np.loadtxt(os.path.join(name,'groundtruth_rect.txt'))
    img_paths = []
    for img_path in glob.glob(os.path.join(name,'img/*')): 
        img_paths.append(img_path)
    img_paths.sort()
    
    for i in xrange(len(boxes)):
        box = boxes[i]
        track_ids[0].append([img_paths[i],[float(box[0]-1),float(box[0]-1+box[2]),float(box[1]-1),float(box[1]-1+box[3])]])
    sequences_info.append(track_ids)
    
    print name,len(img_paths),len(boxes)

/home/leeyh/backup/data/Man 134 134
/home/leeyh/backup/data/CarDark 393 393
/home/leeyh/backup/data/Human6 792 792
/home/leeyh/backup/data/Freeman3 474 460
/home/leeyh/backup/data/Matrix 100 100
/home/leeyh/backup/data/Tiger1 354 354
/home/leeyh/backup/data/CarScale 253 252
/home/leeyh/backup/data/Deer 71 71
/home/leeyh/backup/data/Bird2 99 99
/home/leeyh/backup/data/BlurCar1 742 742
/home/leeyh/backup/data/Lemming 1336 1336
/home/leeyh/backup/data/Skating2-1 473 473
/home/leeyh/backup/data/Trans 124 124
/home/leeyh/backup/data/Boy 602 602
/home/leeyh/backup/data/Jogging-1 307 307
/home/leeyh/backup/data/Toy 271 271
/home/leeyh/backup/data/Diving 231 215
/home/leeyh/backup/data/Car2 913 913
/home/leeyh/backup/data/Football1 81 74
/home/leeyh/backup/data/Jump 122 122
/home/leeyh/backup/data/Skating1 400 400
/home/leeyh/backup/data/Crossing 120 120
/home/leeyh/backup/data/Bolt 350 350
/home/leeyh/backup/data/BasketBall 725 725
/home/leeyh/backup/data/Subway 176 175
/home/leeyh/backup/dat

In [52]:
with open('otb100.txt', 'w') as outfile:
    json.dump(sequences_info, outfile)